# Generate Choice file for all participants

In [7]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame as DF
# Useful to quickly extract csv files
import glob
import os

In [8]:
# Extract Rate data

In [9]:
# Load data all participants
rateNameFrames = glob.glob('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/*_RateData_PFT.csv')
choiceNameFrames = glob.glob('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/*_ChoiceData_PFT.csv')

rateNameFrames.sort()
choiceNameFrames.sort()

In [10]:
rateNameFrames

['/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S1_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S2_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S3_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S4_RateData_PFT.csv']

In [11]:
choiceNameFrames

['/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S1_ChoiceData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S2_ChoiceData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S3_ChoiceData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S4_ChoiceData_PFT.csv']

## Extremes of rating scale in this task : 1 is min, 50 is max

In [43]:
partic_num=len(rateNameFrames) #number of participants

choiceFrameAll = pd.DataFrame( columns = ['trial','leftId','rightId','Frame','endTime','choice','choiceTime','LRat1','RRat1','LRat2','RRat2','LVar1','RVar1','LVar2','RVar2','Part'])

for j in range (partic_num):
    rateFrame = pd.read_csv(rateNameFrames[j])
    choiceFrames = pd.read_csv(choiceNameFrames[j])
    
    if 'ratL' in choiceFrames.columns:
        if 'ratR' in choiceFrames.columns:
            choiceFrames = choiceFrames.drop(['ratL', 'ratR'], axis=1)

    
    idNames = list(rateFrame.picId.unique())
    IdRatings1 = [None]*len(idNames)
    IdRatings2 = [None]*len(idNames)
    IdRatingsVar1 = [None]*len(idNames)
    IdRatingsVar2 = [None]*len(idNames)

    
    #extract average rating for like, dislike
    for i in range(len(idNames)):
        IdRatings1[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 1)].rating.mean()
        IdRatings2[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 2)].rating.mean()
        
        IdRatingsVar1[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 1)].rating.std()
        IdRatingsVar2[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 2)].rating.std()

    rateValues = pd.DataFrame( {'nameId': idNames,'rat1': IdRatings1,'rat2': IdRatings2,'var1': IdRatingsVar1,'var2': IdRatingsVar2,})
    # separate ratings and rating-variability for each frame
    ChoiceLRat1 = [None]*len(choiceFrames)
    ChoiceRRat1 = [None]*len(choiceFrames)
    ChoiceLRat2 = [None]*len(choiceFrames)
    ChoiceRRat2 = [None]*len(choiceFrames)

    ChoiceLVar1 = [None]*len(choiceFrames)
    ChoiceRVar1 = [None]*len(choiceFrames)
    ChoiceLVar2 = [None]*len(choiceFrames)
    ChoiceRVar2 = [None]*len(choiceFrames)
    
    for i in range(len(choiceFrames)):
        
        choiceL = choiceFrames.iloc[i].leftId
        choiceR = choiceFrames.iloc[i].rightId
        
        if any((rateValues['nameId'] == choiceL)) and any((rateValues['nameId'] == choiceR)):
        
            ChoiceLRat1[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].rat1.values[0]
            ChoiceRRat1[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].rat1.values[0]
            ChoiceLRat2[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].rat2.values[0]
            ChoiceRRat2[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].rat2.values[0]
            
            ChoiceLVar1[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].var1.values[0]
            ChoiceRVar1[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].var1.values[0]
            ChoiceLVar2[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].var2.values[0]
            ChoiceRVar2[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].var2.values[0]

    choiceFrames['LRat1'] = ChoiceLRat1
    choiceFrames['RRat1'] = ChoiceRRat1
    choiceFrames['LRat2'] = ChoiceLRat2
    choiceFrames['RRat2'] = ChoiceRRat2 
    
    choiceFrames['LVar1'] = ChoiceLVar1
    choiceFrames['RVar1'] = ChoiceRVar1
    choiceFrames['LVar2'] = ChoiceLVar2
    choiceFrames['RVar2'] = ChoiceRVar2
     
    choiceFrames['Part'] = [j]*len(choiceFrames)    

    choiceFrameAll = pd.concat([choiceFrameAll,choiceFrames],ignore_index=True)
    

# Save file

In [45]:
choiceFrameAll.to_csv('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/All/PFT_Choice_Pilot_All.csv',index = False)

In [44]:
choiceFrameAll

,trial,leftId,rightId,Frame,endTime,choice,choiceTime,LRat1,RRat1,LRat2,RRat2,LVar1,RVar1,LVar2,RVar2,Part
0,0.0,21.0,4.0,2.0,7.508409,1.0,1.174838,50.0,28.5,1.5,20.5,0.000000,3.535534,0.707107,6.363961,0
1,1.0,12.0,14.0,2.0,7.513930,1.0,0.780269,40.5,27.5,10.0,23.0,0.707107,2.121320,5.656854,4.242641,0
2,2.0,20.0,19.0,2.0,7.515296,1.0,1.195317,50.0,8.0,1.0,41.0,0.000000,0.000000,0.000000,12.727922,0
3,3.0,2.0,8.0,2.0,7.514498,1.0,0.247736,44.5,10.5,1.0,50.0,3.535534,3.535534,0.000000,0.000000,0
4,4.0,4.0,21.0,1.0,7.514311,1.0,0.178482,28.5,50.0,20.5,1.5,3.535534,0.000000,6.363961,0.707107,0
5,5.0,12.0,20.0,1.0,7.513606,-1.0,2.181133,40.5,50.0,10.0,1.0,0.707107,0.000000,5.656854,0.000000,0
6,6.0,24.0,21.0,2.0,7.514916,1.0,3.361822,50.0,50.0,25.5,1.5,0.000000,0.000000,34.648232,0.707107,0
7,7.0,2.0,32.0,2.0,7.513895,1.0,2.000413,44.5,31.0,1.0,20.0,3.535534,1.414214,0.000000,0.000000,0
8,8.0,19.0,8.0,1.0,7.514563,1.0,3.298472,8.0,10.5,41.0,50.0,0.000000,3.535534,12.727922,0.000000,0
9,9.0,24.0,21.0,1.0,7.514475,1.0,1.482191,50.0,50.0,25.5,1.5,0.000000,0.000000,34.648232,0.707107,0
